In [1]:
# Необхідні бібліотеки.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt

%matplotlib inline

# Встановлення випадкового початкового числа для відтворюваності
torch.manual_seed(42)

# Перетворення зображення в числа.
transform = transforms.ToTensor()

# Завантаження тренувального та тестового датасетів.
train_data = datasets.MNIST(root='C:/Users/Admin/Downloads/Lesson 54', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='C:/Users/Admin/Downloads/Lesson 54', train=False, download=False, transform=transform)
train_data
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: C:/Users/Admin/Downloads/Lesson 54
    Split: Test
    StandardTransform
Transform: ToTensor()

In [2]:
# Створення завантажувача даних для тренувального датасету.
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)

# Створення завантажувача даних для тестового датасету.
test_loader = DataLoader(test_data, batch_size=500, shuffle=False)

In [3]:
# Параметри моделі.
input_size = 784
hidden_sizes = [120, 84]
output_size = 10

# Створення класу для нейромережі.
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = x.view(-1, input_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# Створення об'єкту нейромережі.
model = Classification()

# Ініціалізація функції помилки та оптимізатора.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
for images, labels in train_loader:
    print('BEFORE [Initial data shape]:', images.size())
    break
print('AFTER [Data after reshaping]:', images.view(100, -1).size())

BEFORE [Initial data shape]: torch.Size([100, 1, 28, 28])
AFTER [Data after reshaping]: torch.Size([100, 784])


In [ ]:
# Тренування моделі.
epochs = 10
train_losses = []
test_losses = []
train_acc = []
test_acc = []

for epoch in range(epochs):
    train_loss = 0.0
    train_accuracy = 0.0

    # Проходження по тренувальним пакетам.
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output.data, 1)
        train_accuracy += (predicted == labels).sum().item()
        train_loss += loss.item()

    train_accuracy /= len(train_data)
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    train_acc.append(train_accuracy)

    # Оцінка моделі на тестових даних.
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        test_accuracy = 0.0
        for images, labels in test_loader:
            output = model(images)
            loss = criterion(output, labels)
            _, predicted = torch.max(output.data, 1)
            test_accuracy += (predicted == labels).sum().item()
            test_loss += loss.item()

        test_accuracy /= len(test_data)
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        test_acc.append(test_accuracy)

    # Виведення прогресу під час тренування.
    print(f'Епоха: {epoch+1}/{epochs}, Помилка тренування: {train_loss:.4f}, Точність тренування: {train_accuracy:.4f}, Помилка тестування: {test_loss:.4f}, Точність тестування: {test_accuracy:.4f}')

# Візуалізація прогресу тренування.
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Train')
plt.plot(range(1, epochs + 1), test_losses, label='Test')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_acc, label='Train')
plt.plot(range(1, epochs + 1), test_acc, label='Test')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

Епоха: 1/10, Помилка тренування: 0.3863, Точність тренування: 0.8913, Помилка тестування: 0.1861, Точність тестування: 0.9457
Епоха: 2/10, Помилка тренування: 0.1568, Точність тренування: 0.9538, Помилка тестування: 0.1227, Точність тестування: 0.9631
Епоха: 3/10, Помилка тренування: 0.1064, Точність тренування: 0.9686, Помилка тестування: 0.1006, Точність тестування: 0.9688
Епоха: 4/10, Помилка тренування: 0.0791, Точність тренування: 0.9764, Помилка тестування: 0.0967, Точність тестування: 0.9692
Епоха: 5/10, Помилка тренування: 0.0617, Точність тренування: 0.9809, Помилка тестування: 0.0779, Точність тестування: 0.9756
Епоха: 6/10, Помилка тренування: 0.0492, Точність тренування: 0.9849, Помилка тестування: 0.0834, Точність тестування: 0.9721
Епоха: 7/10, Помилка тренування: 0.0393, Точність тренування: 0.9881, Помилка тестування: 0.0770, Точність тестування: 0.9769
Епоха: 8/10, Помилка тренування: 0.0334, Точність тренування: 0.9893, Помилка тестування: 0.0786, Точність тестування:

In [ ]:
# Візуалізація випадкового зображення
import random

# Вибір випадкового зображення з тестового датасету
random_image_idx = random.randint(0, len(test_data)-1)
random_image, random_label = test_data[random_image_idx]

# Передача зображення через модель
model.eval()
with torch.no_grad():
    output = model(random_image.unsqueeze(0))
    predicted_label = torch.argmax(output, dim=1).item()

# Відображення зображення та його прогнозованого мітки
plt.figure(figsize=(6, 6))
plt.imshow(random_image.squeeze(), cmap='gray')
plt.title(f'Predicted: {predicted_label}, Actual: {random_label}')
plt.axis('off')
plt.show()

In [ ]:
# Оновлення помилки та точності для тестових даних в кінці кожної епохи
loss = criterion(output, labels)
test_losses.append(loss)
test_correct.append((predicted == labels).sum().item())

# Точність тестових даних для останньої епохи
print(f'Test accuracy: {test_correct[-1] * 100 / len(test_data):.3f}%')

# Збереження та завантаження моделі
torch.save(model, 'model.pkl')
model = torch.load('model.pkl')

# Збереження тільки параметрів моделі
torch.save(model.state_dict(), 'params.pkl')
model.load_state_dict(torch.load('params.pkl'))